#  Convolutional NN it for facial keypoints recognition

Here I will build a convolutional neural network, and train it for the task of facial keypoints recognition. The data are obtained from Kaggle: **LINK**, and consists of **info**.

I will build the CNN using tensorflow **link**. 

##  Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import tensorflow as tf

%matplotlib inline

##  Loading data

In [2]:
training_data_full = pd.read_csv('training.csv')
#training_data.info()

# I will do the splitting here, so that I have a test set that has never been passed 
# through the network.



In [ ]:
# Splitting into training and cross-validation sets. 
training_data, training_cv = train_test_split(training_data_full,
                                              test_size = 50, random_state = 42) #set the state.

In [3]:
images = training_data_full['Image'].apply(lambda str_pic: np.array([int(px) for px in str_pic.split()]))

In [8]:
images = np.vstack(images.iloc[i] for i in range(len(images)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [12]:
keypoints = training_data_full.drop('Image', axis = 1)

In [54]:
x_train, x_cv, keypoints_train, keypoints_cv = train_test_split(images, keypoints, 
                                                                test_size=100, random_state = 42)

In [55]:
def next_batch(batch_size):
    
    sample_indices = np.random.randint(len(x_train), size = batch_size)
    
    images = x_train[sample_indices]
    keypoints = keypoints_train.iloc[sample_indices]
        
    return images, keypoints

# Building the CNN

I will start by using the same architechture I used in the course.For the moment I will use train_test_split to test my network a bit. Eventually this wont be necessary, as the dataset provides a separate test set. 

In [19]:
#Some helping functions.
def init_weights(shape):
    init_random_dist = tf.random_normal(shape, stddev=0.1) # Why this stddev?
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.random_uniform(shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                          strides=[1, 4, 4, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b


In [41]:
# PLACEHOLDERS

x = tf.placeholder(tf.float32, [None, 9216])
keypoints_true = tf.placeholder(tf.float32, [None, 30])
lr = tf.placeholder(tf.float32)

In [42]:
x_images = tf.reshape(x, [-1,96,96,1])

In [43]:
# LAYERS

convo1 = convolutional_layer(x_images, [8,8,1,32]) # 8 x 8 filter, 1 channel in, 32 channels out. SAME padding.
                                            # so output images are also 96 x 96. 
convo1_pool = max_pool_2by2(convo1)   #output of 24 x 24 x 32

In [44]:
convo2 = convolutional_layer(convo1_pool, [4,4,32,64]) # 4x4 filter, 64 outputs. SAME padding.
convo2_pool = max_pool_2by2(convo2) # 6 x 6 x 64

convo2_flat = tf.reshape(convo2_pool,[-1,6*6*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo2_flat,1024))

In [ ]:
"""convo3 = convolutional_layer(convo2_pool, [4,4,64,128]) # 4x4 filter, 64 outputs. SAME padding.
convo3_pool = max_pool_2by2(convo3) # 12 x 12 x 128
convo3_flat = tf.reshape(convo3_pool,[-1,12*12*128])
full_layer_one = tf.nn.relu(normal_full_layer(convo3_flat,1024))"""

In [45]:
# DROPOUT AND OUTPOUT LAYER
drop_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, rate = drop_prob)

keypoints_pred = normal_full_layer(full_one_dropout,30)

In [46]:
# LOSS FUNCTION

masked_loss = tf.reduce_mean(tf.square(
        tf.boolean_mask(keypoints_pred-keypoints_true, tf.is_finite(keypoints_true) )
    ))


optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train = optimizer.minimize(masked_loss)

In [47]:
# INITIALIZER

init = tf.global_variables_initializer()

In [48]:
saver = tf.train.Saver()

In [49]:
counter = 0

In [50]:
keypoints_mean = keypoints_train.mean()

In [51]:
# Computing error from the mean in order to have a reference. This is the mark I have to beat. 

mean_keypoints_mse = ((keypoints_cv-keypoints_mean)**2).sum(axis = 1).mean()
mean_keypoints_mse

210.69596922581684

In [57]:
num_steps = 5000

with tf.Session() as sess:
        
    sess.run(init)
    #saver.restore(sess, "./saved_models/keypoints_cnn_3")
    train_losses = []
    cv_losses = []
    for iteration in range(num_steps+1):
        
        x_batch, keypoints_batch = next_batch(50)
        
        _ , train_loss = sess.run([train, masked_loss], 
                                  feed_dict={x: x_batch, keypoints_true: keypoints_batch,
                                             drop_prob: 0.5, lr:0.0005})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        
        #if (iteration%20 == 0) and (iteration >0):
            #print(':')
        
        if iteration%100 == 0:
            
            cv_loss = sess.run(masked_loss,feed_dict={x:x_cv,keypoints_true:keypoints_cv,
                                                      drop_prob:0})
            
            print('Currently on step {}'.format(iteration))
            print('Train MSE: ')
            print(train_loss, '\n')
            print('CV MSE:')
            # Test the Train Model
            print(cv_loss)
            print('\n')
            
            train_losses.append(train_loss)
            cv_losses.append(cv_loss)
            print('train_loss: ')
            print( train_losses)
            print('cv_loss: ')
            print(cv_losses, '\n')
            
    
    saver.save(sess, "./saved_models/keypoints_cnn_" + str(counter) )
    #saver.save(sess, "./saved_models/keypoints_cnn_3"  )
    #counter +=1

Currently on step 0
Train MSE: 
8248813.0 

CV MSE:
1223629.2


train_loss: 
[8248813.0]
cv_loss: 
[1223629.2] 

Currently on step 100
Train MSE: 
5215.374 

CV MSE:
2398.4114


train_loss: 
[8248813.0, 5215.374]
cv_loss: 
[1223629.2, 2398.4114] 

Currently on step 200
Train MSE: 
1627.257 

CV MSE:
852.15576


train_loss: 
[8248813.0, 5215.374, 1627.257]
cv_loss: 
[1223629.2, 2398.4114, 852.15576] 

Currently on step 300
Train MSE: 
963.4868 

CV MSE:
402.28214


train_loss: 
[8248813.0, 5215.374, 1627.257, 963.4868]
cv_loss: 
[1223629.2, 2398.4114, 852.15576, 402.28214] 

Currently on step 400
Train MSE: 
538.09033 

CV MSE:
337.3574


train_loss: 
[8248813.0, 5215.374, 1627.257, 963.4868, 538.09033]
cv_loss: 
[1223629.2, 2398.4114, 852.15576, 402.28214, 337.3574] 

Currently on step 500
Train MSE: 
696.5594 

CV MSE:
241.59712


train_loss: 
[8248813.0, 5215.374, 1627.257, 963.4868, 538.09033, 696.5594]
cv_loss: 
[1223629.2, 2398.4114, 852.15576, 402.28214, 337.3574, 241.59712] 



KeyboardInterrupt: 

In [ ]:
counter

In [ ]:
with tf.Session() as sess:
    
    # Use your Saver instance to restore your saved rnn time series model
    saver.restore(sess, "./saved_models/keypoints_cnn_1")

    # Create a numpy array for your genreative seed from the last 12 months of the 
    # training set data. Hint: Just use tail(12) and then pass it to an np.array
        
    predictions = sess.run(keypoints_pred, feed_dict= {x:x_cv_small, hold_prob:1.0 })
    

In [ ]:
predictions.shape

In [ ]:
x_cv_small.shape

In [ ]:
fig, axes = plt.subplots(3, 3, gridspec_kw = dict(hspace = .05, wspace = .05), 
                         figsize=(10,10))

mean_x_points = [keypoints_mean[j] for j in range(0,30,2)]
mean_y_points = [keypoints_mean[j+1] for j in range(0,30,2)]

for i, ax in enumerate(axes.flat):

    ax.axis('off')
    # Plotting the faces
    ax.imshow(x_cv_small[i].reshape((96,96)),cmap='gist_gray')

 # Obtaining keypoints positions. x and y coordinates are even and odd indices respectively. 
    x_points = [predictions[i][j] for j in range(0,30,2)]
    y_points = [predictions[i][j+1] for j in range(0,30,2)]
      
    #plotting keypoints
    ax.plot(x_points, y_points, 'ro', markerfacecolor = 'none')    
  # Including mean keypoints
       
    ax.plot(mean_x_points, mean_y_points, 'b+', markerfacecolor = 'none')  